. Import libraries

In [275]:
import pandas as pd
import os
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point, Polygon
import csv

### Bornes

In [276]:
data = pd.read_csv("raw_data/borne_fontaine.csv")

In [277]:
data

,MUNID,CODEID,TYPE,NO_CIVIQUE,GENERIQUE,LIAISON,SPECIFIQUE,DIRECTION,ETIQUETTE,PROPRIO,ENTITEID,GEOM
0,81017,266,Eau potable,NaN,Rue,NaN,Émard,NaN,1840-1,Ville,{863C7712-F50F-496B-8E31-92D0165FA301},POINT (-75.738712 45.500612)
1,81017,268,Eau potable,NaN,Rue,de,Nantes,NaN,1279-1,Ville,{961B3EB9-91E6-462C-9E7E-D65220DF70E9},POINT (-75.722305 45.47477)
2,81017,269,Eau potable,NaN,Rue,de,Monte-Carlo,NaN,2241-2,Ville,{C35C6749-6B6C-4FF3-8DE9-4DAE02B5A33A},POINT (-75.735754 45.486355)
3,81017,270,Eau potable,NaN,Rue,de,Villefranche,NaN,2594-2,Ville,{1A8BE66A-C214-4BD8-B874-75D7771AB262},POINT (-75.737537 45.49035)
4,81017,271,Eau potable,NaN,Rue,du,Sommelier,NaN,2453-1,Ville,{D9611000-D5B3-442D-B70C-00468C9088AE},POINT (-75.749615 45.497777)
...,...,...,...,...,...,...,...,...,...,...,...,...
8072,81017,9619,Eau potable,NaN,Avenue,des,Entreprises,NaN,1846-12,Privé,{74FDF055-2896-4B26-A28C-1EF2F88D57D5},POINT (-75.651069 45.512558)
8073,81017,9618,Eau potable,NaN,Avenue,des,Entreprises,NaN,1846-11,Privé,{DF0FF676-7449-469C-91A0-58A41C43CC78},POINT (-75.650557 45.511914)
8074,81017,9621,Eau potable,NaN,Boulevard,du,Plateau,NaN,1305-100,Ville,{2FE3CB18-9798-43CF-9F89-6F3305228B27},POINT (-75.809197 45.422396)
8075,81017,9622,Eau potable,NaN,Boulevard,du,Plateau,NaN,1305-101,Ville,{7B44A5F5-7924-4BFE-975F-8A6C73547A22},POINT (-75.80901 45.421887)


In [278]:
data.dtypes

MUNID           int64
CODEID          int64
TYPE           object
NO_CIVIQUE    float64
GENERIQUE      object
LIAISON        object
SPECIFIQUE     object
DIRECTION      object
ETIQUETTE      object
PROPRIO        object
ENTITEID       object
GEOM           object
dtype: object

In [279]:
data["NO_CIVIQUE"].isna().sum()

8077

In [280]:
data["DIRECTION"].isna().sum()

7713

In [281]:
data[data["DIRECTION"].notna()]

,MUNID,CODEID,TYPE,NO_CIVIQUE,GENERIQUE,LIAISON,SPECIFIQUE,DIRECTION,ETIQUETTE,PROPRIO,ENTITEID,GEOM
165,81017,786,Eau potable,NaN,Boulevard,NaN,Saint-René,Ouest,2528-9,Ville,{164B27CA-9E10-4F68-BD07-323A3D17517B},POINT (-75.699139 45.488635)
166,81017,788,Eau potable,NaN,Boulevard,NaN,Saint-René,Ouest,2528-5,Ville,{C43321DE-C3EA-4D9B-BFD5-9D85524E67EA},POINT (-75.700785 45.488507)
169,81017,792,Eau potable,NaN,Boulevard,NaN,Saint-René,Ouest,2528-17,Ville,{21323023-5188-44DC-BAC6-0046512911D2},POINT (-75.688509 45.489468)
170,81017,794,Eau potable,NaN,Boulevard,NaN,Saint-René,Ouest,2528-14,Ville,{616B079C-BA49-418E-9FBF-521D9F993C29},POINT (-75.691912 45.489199)
245,81017,884,Eau potable,NaN,Boulevard,NaN,Saint-René,Ouest,2528-4,Ville,{165BDB3B-6DF1-4F0A-ABFA-62E33973186B},POINT (-75.702428 45.488382)
...,...,...,...,...,...,...,...,...,...,...,...,...
7937,81017,9475,Eau potable,NaN,Boulevard,NaN,Saint-René,Ouest,2528-37,Ville,{115860F9-6B7B-4291-9A0D-84E15269B2BB},POINT (-75.685976 45.491514)
7938,81017,9474,Eau potable,NaN,Boulevard,NaN,Saint-René,Ouest,2528-36,Ville,{8CE37E17-18CC-4942-ABF2-C37F5E4B0F9E},POINT (-75.686681 45.491075)
7972,81017,9513,Eau potable,NaN,Boulevard,NaN,Saint-René,Est,2527-57,Ville,{791411DE-719C-401F-9DAC-27DCD053037B},POINT (-75.621363 45.499445)
8031,81017,9572,Eau potable,NaN,Boulevard,NaN,Saint-René,Est,2527-58,Ville,{24DAE2AB-65C2-43CC-A835-C63CFD66E44A},POINT (-75.615157 45.49926)


In [282]:
data["DIRECTION"] = data["DIRECTION"].fillna("Non attribué")

In [283]:
data = data.drop(columns=["NO_CIVIQUE"])

In [284]:
data.columns = data.columns.str.lower()

In [285]:
data["liaison"].isna().sum()

4641

In [286]:
data["liaison"] = data["liaison"].fillna("")

In [287]:
data["lieu"] = data[["generique", "liaison", "specifique"]].astype(str).agg(" ".join, axis=1)

In [288]:
data["lieu"] = data["lieu"].str.replace(r"\s+", " ", regex=True).str.strip()

In [289]:
data["lieu"]

0                    Rue Émard
1                Rue de Nantes
2           Rue de Monte-Carlo
3          Rue de Villefranche
4             Rue du Sommelier
                 ...          
8072    Avenue des Entreprises
8073    Avenue des Entreprises
8074      Boulevard du Plateau
8075      Boulevard du Plateau
8076      Boulevard du Plateau
Name: lieu, Length: 8077, dtype: object

In [290]:
data = data.drop(columns=["generique","liaison","specifique"])

In [291]:
data = data[["munid", "codeid", "type", "lieu", "etiquette", "proprio", "entiteid", "geom"]]

In [292]:
data["type"].value_counts()

type
Eau potable    8070
Eau brute         7
Name: count, dtype: int64

In [293]:
data["proprio"].value_counts()

proprio
Ville    7451
Privé     617
N/D         4
CCN         2
STO         2
MTQ         1
Name: count, dtype: int64

### Propriétaires
#### Ville → Infrastructures appartenant à la Ville de Gatineau.

#### Privé → Propriété privée (entreprises, citoyens, promoteurs).

#### N/D → Non Disponible ou Non Défini → propriétaire inconnu / donnée manquante.

#### CCN → Commission de la capitale nationale (organisme fédéral responsable de certains terrains, parcs, sentiers et routes de la région de la capitale nationale, incluant Gatineau et Ottawa).

#### STO → Société de transport de l’Outaouais (transport en commun de Gatineau : autobus, infrastructures connexes).

#### MTQ → Ministère des Transports du Québec (autoroutes, routes numérotées et certaines infrastructures routières).

In [294]:
data.duplicated().sum()

0

In [295]:
data.isna().sum()

munid        0
codeid       0
type         0
lieu         0
etiquette    0
proprio      0
entiteid     0
geom         0
dtype: int64

In [296]:
data.isnull().sum()

munid        0
codeid       0
type         0
lieu         0
etiquette    0
proprio      0
entiteid     0
geom         0
dtype: int64

In [297]:
data = data.copy()

data[["longitude", "latitude"]] = data["geom"].str.extract(
    r"POINT \((-?\d+\.\d+)\s+(-?\d+\.\d+)\)"
)

data.loc[:, "longitude"] = data["longitude"].astype(float)
data.loc[:, "latitude"]  = data["latitude"].astype(float)

In [298]:
folder = "dataset_cleaned"
os.makedirs(folder, exist_ok=True)
path = os.path.join(folder, "borne_clean.csv")
data.to_csv(path, index=False, encoding="utf-8")

print(f"File saved in: {path}")


File saved in: dataset_cleaned\borne_clean.csv


In [299]:
data

,munid,codeid,type,lieu,etiquette,proprio,entiteid,geom,longitude,latitude
0,81017,266,Eau potable,Rue Émard,1840-1,Ville,{863C7712-F50F-496B-8E31-92D0165FA301},POINT (-75.738712 45.500612),-75.738712,45.500612
1,81017,268,Eau potable,Rue de Nantes,1279-1,Ville,{961B3EB9-91E6-462C-9E7E-D65220DF70E9},POINT (-75.722305 45.47477),-75.722305,45.47477
2,81017,269,Eau potable,Rue de Monte-Carlo,2241-2,Ville,{C35C6749-6B6C-4FF3-8DE9-4DAE02B5A33A},POINT (-75.735754 45.486355),-75.735754,45.486355
3,81017,270,Eau potable,Rue de Villefranche,2594-2,Ville,{1A8BE66A-C214-4BD8-B874-75D7771AB262},POINT (-75.737537 45.49035),-75.737537,45.49035
4,81017,271,Eau potable,Rue du Sommelier,2453-1,Ville,{D9611000-D5B3-442D-B70C-00468C9088AE},POINT (-75.749615 45.497777),-75.749615,45.497777
...,...,...,...,...,...,...,...,...,...,...
8072,81017,9619,Eau potable,Avenue des Entreprises,1846-12,Privé,{74FDF055-2896-4B26-A28C-1EF2F88D57D5},POINT (-75.651069 45.512558),-75.651069,45.512558
8073,81017,9618,Eau potable,Avenue des Entreprises,1846-11,Privé,{DF0FF676-7449-469C-91A0-58A41C43CC78},POINT (-75.650557 45.511914),-75.650557,45.511914
8074,81017,9621,Eau potable,Boulevard du Plateau,1305-100,Ville,{2FE3CB18-9798-43CF-9F89-6F3305228B27},POINT (-75.809197 45.422396),-75.809197,45.422396
8075,81017,9622,Eau potable,Boulevard du Plateau,1305-101,Ville,{7B44A5F5-7924-4BFE-975F-8A6C73547A22},POINT (-75.80901 45.421887),-75.80901,45.421887


### Cleaning borne for Postgres

In [300]:
data_borne = pd.read_csv("dataset_cleaned/borne_clean.csv")

In [301]:
data_borne

,munid,codeid,type,lieu,etiquette,proprio,entiteid,geom,longitude,latitude
0,81017,266,Eau potable,Rue Émard,1840-1,Ville,{863C7712-F50F-496B-8E31-92D0165FA301},POINT (-75.738712 45.500612),-75.738712,45.500612
1,81017,268,Eau potable,Rue de Nantes,1279-1,Ville,{961B3EB9-91E6-462C-9E7E-D65220DF70E9},POINT (-75.722305 45.47477),-75.722305,45.474770
2,81017,269,Eau potable,Rue de Monte-Carlo,2241-2,Ville,{C35C6749-6B6C-4FF3-8DE9-4DAE02B5A33A},POINT (-75.735754 45.486355),-75.735754,45.486355
3,81017,270,Eau potable,Rue de Villefranche,2594-2,Ville,{1A8BE66A-C214-4BD8-B874-75D7771AB262},POINT (-75.737537 45.49035),-75.737537,45.490350
4,81017,271,Eau potable,Rue du Sommelier,2453-1,Ville,{D9611000-D5B3-442D-B70C-00468C9088AE},POINT (-75.749615 45.497777),-75.749615,45.497777
...,...,...,...,...,...,...,...,...,...,...
8072,81017,9619,Eau potable,Avenue des Entreprises,1846-12,Privé,{74FDF055-2896-4B26-A28C-1EF2F88D57D5},POINT (-75.651069 45.512558),-75.651069,45.512558
8073,81017,9618,Eau potable,Avenue des Entreprises,1846-11,Privé,{DF0FF676-7449-469C-91A0-58A41C43CC78},POINT (-75.650557 45.511914),-75.650557,45.511914
8074,81017,9621,Eau potable,Boulevard du Plateau,1305-100,Ville,{2FE3CB18-9798-43CF-9F89-6F3305228B27},POINT (-75.809197 45.422396),-75.809197,45.422396
8075,81017,9622,Eau potable,Boulevard du Plateau,1305-101,Ville,{7B44A5F5-7924-4BFE-975F-8A6C73547A22},POINT (-75.80901 45.421887),-75.809010,45.421887


In [302]:
# drop the column munid
data_borne.drop(columns="munid", inplace= True)

In [303]:
data_borne

,codeid,type,lieu,etiquette,proprio,entiteid,geom,longitude,latitude
0,266,Eau potable,Rue Émard,1840-1,Ville,{863C7712-F50F-496B-8E31-92D0165FA301},POINT (-75.738712 45.500612),-75.738712,45.500612
1,268,Eau potable,Rue de Nantes,1279-1,Ville,{961B3EB9-91E6-462C-9E7E-D65220DF70E9},POINT (-75.722305 45.47477),-75.722305,45.474770
2,269,Eau potable,Rue de Monte-Carlo,2241-2,Ville,{C35C6749-6B6C-4FF3-8DE9-4DAE02B5A33A},POINT (-75.735754 45.486355),-75.735754,45.486355
3,270,Eau potable,Rue de Villefranche,2594-2,Ville,{1A8BE66A-C214-4BD8-B874-75D7771AB262},POINT (-75.737537 45.49035),-75.737537,45.490350
4,271,Eau potable,Rue du Sommelier,2453-1,Ville,{D9611000-D5B3-442D-B70C-00468C9088AE},POINT (-75.749615 45.497777),-75.749615,45.497777
...,...,...,...,...,...,...,...,...,...
8072,9619,Eau potable,Avenue des Entreprises,1846-12,Privé,{74FDF055-2896-4B26-A28C-1EF2F88D57D5},POINT (-75.651069 45.512558),-75.651069,45.512558
8073,9618,Eau potable,Avenue des Entreprises,1846-11,Privé,{DF0FF676-7449-469C-91A0-58A41C43CC78},POINT (-75.650557 45.511914),-75.650557,45.511914
8074,9621,Eau potable,Boulevard du Plateau,1305-100,Ville,{2FE3CB18-9798-43CF-9F89-6F3305228B27},POINT (-75.809197 45.422396),-75.809197,45.422396
8075,9622,Eau potable,Boulevard du Plateau,1305-101,Ville,{7B44A5F5-7924-4BFE-975F-8A6C73547A22},POINT (-75.80901 45.421887),-75.809010,45.421887


In [304]:
data_borne["codeid"].duplicated().sum()

0

In [305]:
data_borne.rename(columns={"codeid": "borneid"}, inplace=True)

In [306]:
#map the sector which corresponding to the latitude and longitude
import geopandas as gpd

url = "https://www.gatineau.ca/upload/donneesouvertes/DECOUPAGE_ADMINISTRATIF.json" 
gdf = gpd.read_file(url)
print(gdf.head())


   MUNID CODEID            NOM      TYPE ABREV          AIRE     PERIM  \
0  81017      5     Buckingham  Ex ville     B  1.608119e+07  18214.35   
1  81017     10  Masson-Angers  Ex ville     M  6.453902e+07  46386.20   
2  81017     15       Gatineau  Ex ville     G  1.585884e+08  58086.51   
3  81017     20           Hull  Ex ville     H  4.133173e+07  32973.84   
4  81017     25         Aylmer  Ex ville     A  1.000886e+08  42578.92   

                                 ENTITEID  \
0  {0DE4611D-2A9E-4B26-A1B6-72A9C3DD3DBD}   
1  {96AAFDBC-9949-41EF-8D00-ECE15E302886}   
2  {07DBC407-0068-4AEE-8F5E-51A985940380}   
3  {36908290-4C89-41DB-A00A-C29EB4CFB9A8}   
4  {5C1FA124-6734-44C6-AB31-23B28DC0EF91}   

                                            geometry  
0  POLYGON ((-75.41524 45.59898, -75.41522 45.598...  
1  POLYGON ((-75.35372 45.5957, -75.35272 45.5878...  
2  POLYGON ((-75.53744 45.57731, -75.5331 45.5479...  
3  POLYGON ((-75.77524 45.48664, -75.7751 45.4862...  
4  POLYGO

In [307]:
from shapely.geometry import Point

# Transformer en GeoDataFrame
geometry = [Point(xy) for xy in zip(data_borne["longitude"], data_borne["latitude"])]
gdf_points = gpd.GeoDataFrame(data_borne, geometry=geometry, crs="EPSG:4326")

In [308]:
gdf = gdf.to_crs("EPSG:4326")

In [309]:
geo = gpd.sjoin(gdf_points, gdf, how="left", predicate="within")

In [310]:
geo

,borneid,type,lieu,etiquette,proprio,entiteid,geom,longitude,latitude,geometry,index_right,MUNID,CODEID,NOM,TYPE,ABREV,AIRE,PERIM,ENTITEID
0,266,Eau potable,Rue Émard,1840-1,Ville,{863C7712-F50F-496B-8E31-92D0165FA301},POINT (-75.738712 45.500612),-75.738712,45.500612,POINT (-75.73871 45.50061),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
1,268,Eau potable,Rue de Nantes,1279-1,Ville,{961B3EB9-91E6-462C-9E7E-D65220DF70E9},POINT (-75.722305 45.47477),-75.722305,45.474770,POINT (-75.7223 45.47477),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
2,269,Eau potable,Rue de Monte-Carlo,2241-2,Ville,{C35C6749-6B6C-4FF3-8DE9-4DAE02B5A33A},POINT (-75.735754 45.486355),-75.735754,45.486355,POINT (-75.73575 45.48636),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
3,270,Eau potable,Rue de Villefranche,2594-2,Ville,{1A8BE66A-C214-4BD8-B874-75D7771AB262},POINT (-75.737537 45.49035),-75.737537,45.490350,POINT (-75.73754 45.49035),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
4,271,Eau potable,Rue du Sommelier,2453-1,Ville,{D9611000-D5B3-442D-B70C-00468C9088AE},POINT (-75.749615 45.497777),-75.749615,45.497777,POINT (-75.74962 45.49778),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8072,9619,Eau potable,Avenue des Entreprises,1846-12,Privé,{74FDF055-2896-4B26-A28C-1EF2F88D57D5},POINT (-75.651069 45.512558),-75.651069,45.512558,POINT (-75.65107 45.51256),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
8073,9618,Eau potable,Avenue des Entreprises,1846-11,Privé,{DF0FF676-7449-469C-91A0-58A41C43CC78},POINT (-75.650557 45.511914),-75.650557,45.511914,POINT (-75.65056 45.51191),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
8074,9621,Eau potable,Boulevard du Plateau,1305-100,Ville,{2FE3CB18-9798-43CF-9F89-6F3305228B27},POINT (-75.809197 45.422396),-75.809197,45.422396,POINT (-75.8092 45.4224),4.0,81017.0,25,Aylmer,Ex ville,A,1.000886e+08,42578.92,{5C1FA124-6734-44C6-AB31-23B28DC0EF91}
8075,9622,Eau potable,Boulevard du Plateau,1305-101,Ville,{7B44A5F5-7924-4BFE-975F-8A6C73547A22},POINT (-75.80901 45.421887),-75.809010,45.421887,POINT (-75.80901 45.42189),4.0,81017.0,25,Aylmer,Ex ville,A,1.000886e+08,42578.92,{5C1FA124-6734-44C6-AB31-23B28DC0EF91}


In [311]:
geo.columns

Index(['borneid', 'type', 'lieu', 'etiquette', 'proprio', 'entiteid', 'geom',
       'longitude', 'latitude', 'geometry', 'index_right', 'MUNID', 'CODEID',
       'NOM', 'TYPE', 'ABREV', 'AIRE', 'PERIM', 'ENTITEID'],
      dtype='object')

In [312]:
geo = geo.drop(columns=[
    'geometry',
    'index_right',
    'MUNID',
    'NOM',
    'TYPE',
    'ABREV',
    'AIRE',
    'PERIM',
    'ENTITEID'
])


In [313]:
geo.columns

Index(['borneid', 'type', 'lieu', 'etiquette', 'proprio', 'entiteid', 'geom',
       'longitude', 'latitude', 'CODEID'],
      dtype='object')

In [314]:
geo_dict = {
    "borneid": "borneid",
    "type": "type",
    "lieu": "lieu",
    'etiquette': 'etiquette',
    "proprio": "proprietaire",
    "entiteid": "entiteid",
    "geom": "geom",
    "longitude": "longitude",
    "latitude": "latitude",
    "CODEID": "decoupageid"
}

geo = geo.rename(columns=geo_dict)

In [315]:
geo

,borneid,type,lieu,etiquette,proprietaire,entiteid,geom,longitude,latitude,decoupageid
0,266,Eau potable,Rue Émard,1840-1,Ville,{863C7712-F50F-496B-8E31-92D0165FA301},POINT (-75.738712 45.500612),-75.738712,45.500612,15
1,268,Eau potable,Rue de Nantes,1279-1,Ville,{961B3EB9-91E6-462C-9E7E-D65220DF70E9},POINT (-75.722305 45.47477),-75.722305,45.474770,15
2,269,Eau potable,Rue de Monte-Carlo,2241-2,Ville,{C35C6749-6B6C-4FF3-8DE9-4DAE02B5A33A},POINT (-75.735754 45.486355),-75.735754,45.486355,15
3,270,Eau potable,Rue de Villefranche,2594-2,Ville,{1A8BE66A-C214-4BD8-B874-75D7771AB262},POINT (-75.737537 45.49035),-75.737537,45.490350,15
4,271,Eau potable,Rue du Sommelier,2453-1,Ville,{D9611000-D5B3-442D-B70C-00468C9088AE},POINT (-75.749615 45.497777),-75.749615,45.497777,15
...,...,...,...,...,...,...,...,...,...,...
8072,9619,Eau potable,Avenue des Entreprises,1846-12,Privé,{74FDF055-2896-4B26-A28C-1EF2F88D57D5},POINT (-75.651069 45.512558),-75.651069,45.512558,15
8073,9618,Eau potable,Avenue des Entreprises,1846-11,Privé,{DF0FF676-7449-469C-91A0-58A41C43CC78},POINT (-75.650557 45.511914),-75.650557,45.511914,15
8074,9621,Eau potable,Boulevard du Plateau,1305-100,Ville,{2FE3CB18-9798-43CF-9F89-6F3305228B27},POINT (-75.809197 45.422396),-75.809197,45.422396,25
8075,9622,Eau potable,Boulevard du Plateau,1305-101,Ville,{7B44A5F5-7924-4BFE-975F-8A6C73547A22},POINT (-75.80901 45.421887),-75.809010,45.421887,25


In [316]:
geo.duplicated().sum()

0

In [317]:
geo.isna().sum()

borneid         0
type            0
lieu            0
etiquette       0
proprietaire    0
entiteid        0
geom            0
longitude       0
latitude        0
decoupageid     5
dtype: int64

In [318]:
geo[geo["decoupageid"].isna()]

,borneid,type,lieu,etiquette,proprietaire,entiteid,geom,longitude,latitude,decoupageid
4725,6705,Eau potable,Rue Bélanger,1580-8,Privé,{EF4D8046-4C86-4D92-934D-C958175ABCCF},POINT (-75.411873 45.597486),-75.411873,45.597486,NaN
5128,3329,Eau potable,Chemin Belter,1587-1,Ville,{E796E3C3-A6ED-48AB-A27A-BF49DA92279C},POINT (-75.406519 45.593615),-75.406519,45.593615,NaN
5129,3330,Eau potable,Rue Joseph,2014-8,Ville,{EBC720A2-95FC-4A14-86C9-8BEA542561AF},POINT (-75.406981 45.59428),-75.406981,45.594280,NaN
5863,7746,Eau potable,Rue Bélanger,1580-9,Privé,{45C61FF8-90B5-4581-91A1-1B566FF08785},POINT (-75.413007 45.595222),-75.413007,45.595222,NaN
7686,9195,Eau potable,Boulevard Saint-Joseph,2507-70,Ville,{D1A44620-91B5-4B06-A238-B2F8883A1BE2},POINT (-75.745108 45.47994),-75.745108,45.479940,NaN


In [319]:
geo.loc[geo["lieu"] == "Rue Bélanger", "decoupageid"] = 5
geo.loc[geo["lieu"] == "Chemin Belter", "decoupageid"] = 15
geo.loc[geo["lieu"] == "Rue Joseph", "decoupageid"] = 5
geo.loc[geo["lieu"] == "Boulevard Saint-Joseph", "decoupageid"] = 20


In [320]:
geo.dtypes

borneid           int64
type             object
lieu             object
etiquette        object
proprietaire     object
entiteid         object
geom             object
longitude       float64
latitude        float64
decoupageid      object
dtype: object

In [321]:
geo['decoupageid'] = pd.to_numeric(geo['decoupageid'], errors='coerce').astype('Int64')

In [322]:
geo

,borneid,type,lieu,etiquette,proprietaire,entiteid,geom,longitude,latitude,decoupageid
0,266,Eau potable,Rue Émard,1840-1,Ville,{863C7712-F50F-496B-8E31-92D0165FA301},POINT (-75.738712 45.500612),-75.738712,45.500612,15
1,268,Eau potable,Rue de Nantes,1279-1,Ville,{961B3EB9-91E6-462C-9E7E-D65220DF70E9},POINT (-75.722305 45.47477),-75.722305,45.474770,15
2,269,Eau potable,Rue de Monte-Carlo,2241-2,Ville,{C35C6749-6B6C-4FF3-8DE9-4DAE02B5A33A},POINT (-75.735754 45.486355),-75.735754,45.486355,15
3,270,Eau potable,Rue de Villefranche,2594-2,Ville,{1A8BE66A-C214-4BD8-B874-75D7771AB262},POINT (-75.737537 45.49035),-75.737537,45.490350,15
4,271,Eau potable,Rue du Sommelier,2453-1,Ville,{D9611000-D5B3-442D-B70C-00468C9088AE},POINT (-75.749615 45.497777),-75.749615,45.497777,15
...,...,...,...,...,...,...,...,...,...,...
8072,9619,Eau potable,Avenue des Entreprises,1846-12,Privé,{74FDF055-2896-4B26-A28C-1EF2F88D57D5},POINT (-75.651069 45.512558),-75.651069,45.512558,15
8073,9618,Eau potable,Avenue des Entreprises,1846-11,Privé,{DF0FF676-7449-469C-91A0-58A41C43CC78},POINT (-75.650557 45.511914),-75.650557,45.511914,15
8074,9621,Eau potable,Boulevard du Plateau,1305-100,Ville,{2FE3CB18-9798-43CF-9F89-6F3305228B27},POINT (-75.809197 45.422396),-75.809197,45.422396,25
8075,9622,Eau potable,Boulevard du Plateau,1305-101,Ville,{7B44A5F5-7924-4BFE-975F-8A6C73547A22},POINT (-75.80901 45.421887),-75.809010,45.421887,25


In [323]:
geo.columns

Index(['borneid', 'type', 'lieu', 'etiquette', 'proprietaire', 'entiteid',
       'geom', 'longitude', 'latitude', 'decoupageid'],
      dtype='object')

In [324]:
geo.to_csv("pg_cleaned/borne_geo.csv",index=False)

### Lieux Publiques

In [325]:
data2 = pd.read_csv("raw_data/lieu_publique.csv")

In [326]:
data2.head()

,MUNID,CODEID,TYPE,NO_CIVIQUE,GENERIQUE,LIAISON,SPECIFIQUE,DIRECTION,ADR_COMPLE,NOM_TOPOGR,COTE,SECTION,ENTITEID,GEOM
0,81017,160,Édifice fédéral,NaN,NaN,NaN,NaN,NaN,625 Boulevard du Carrefour,Archives Nationales,NaN,NaN,{C22FB94A-24C3-4F58-8375-231468598ED2},POINT (-75.676786 45.487452)
1,81017,79,Résidence pour personnes âgées,NaN,NaN,NaN,NaN,NaN,60 Rue de la Futaie,Résidence Cité-Jardin,NaN,NaN,{88DE67A5-88F2-47D7-B4AA-BEDC8049F255},POINT (-75.68749 45.486385)
2,81017,570,Cinéma,NaN,NaN,NaN,NaN,NaN,120 Boulevard de l'Hôpital,Cinéma 9,NaN,NaN,{C1DB53E3-86E2-4BF5-ADE1-EEA62F3DB816},POINT (-75.686799 45.481)
3,81017,1162,Autres services médicaux et de santé,NaN,NaN,NaN,NaN,NaN,25 Rue Saint-François,Centre de réadaptation en dépendance,NaN,NaN,{C640A131-656C-4A21-AE93-D6120CDE0A0A},POINT (-75.748249 45.422431)
4,81017,1163,C.H.S.L.D.,NaN,NaN,NaN,NaN,NaN,134 Rue Jean-René-Monette,Centre d'hébergement Bon Séjour,NaN,NaN,{52C9F9B7-3721-4204-A13B-388828278AD5},POINT (-75.647589 45.481621)


In [327]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1295 entries, 0 to 1294
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MUNID       1295 non-null   int64  
 1   CODEID      1295 non-null   int64  
 2   TYPE        1295 non-null   object 
 3   NO_CIVIQUE  0 non-null      float64
 4   GENERIQUE   0 non-null      float64
 5   LIAISON     0 non-null      float64
 6   SPECIFIQUE  0 non-null      float64
 7   DIRECTION   0 non-null      float64
 8   ADR_COMPLE  1294 non-null   object 
 9   NOM_TOPOGR  1295 non-null   object 
 10  COTE        0 non-null      float64
 11  SECTION     0 non-null      float64
 12  ENTITEID    1295 non-null   object 
 13  GEOM        1295 non-null   object 
dtypes: float64(7), int64(2), object(5)
memory usage: 141.8+ KB


In [328]:
data2.shape

(1295, 14)

In [329]:
print(data2["GENERIQUE"].isna().sum())  
print(data2["LIAISON"].isna().sum())
print(data2["SPECIFIQUE"].isna().sum())
print(data2["NO_CIVIQUE"].isna().sum())
print(data2["DIRECTION"].isna().sum())
print(data2["COTE"].isna().sum())
print(data2["SECTION"].isna().sum())
print(data2["ENTITEID"].isna().sum())

1295
1295
1295
1295
1295
1295
1295
0


In [330]:
data2 = data2.drop(columns=["NO_CIVIQUE","GENERIQUE","LIAISON","SPECIFIQUE","DIRECTION","COTE","SECTION"])

In [331]:
data2.columns = data2.columns.str.lower()

In [332]:
data2

,munid,codeid,type,adr_comple,nom_topogr,entiteid,geom
0,81017,160,Édifice fédéral,625 Boulevard du Carrefour,Archives Nationales,{C22FB94A-24C3-4F58-8375-231468598ED2},POINT (-75.676786 45.487452)
1,81017,79,Résidence pour personnes âgées,60 Rue de la Futaie,Résidence Cité-Jardin,{88DE67A5-88F2-47D7-B4AA-BEDC8049F255},POINT (-75.68749 45.486385)
2,81017,570,Cinéma,120 Boulevard de l'Hôpital,Cinéma 9,{C1DB53E3-86E2-4BF5-ADE1-EEA62F3DB816},POINT (-75.686799 45.481)
3,81017,1162,Autres services médicaux et de santé,25 Rue Saint-François,Centre de réadaptation en dépendance,{C640A131-656C-4A21-AE93-D6120CDE0A0A},POINT (-75.748249 45.422431)
4,81017,1163,C.H.S.L.D.,134 Rue Jean-René-Monette,Centre d'hébergement Bon Séjour,{52C9F9B7-3721-4204-A13B-388828278AD5},POINT (-75.647589 45.481621)
...,...,...,...,...,...,...,...
1290,81017,1698,Autres infrastructure,"97, boulevard Saint-Raymond",Planche à roulettes,{F815E3DF-1E6E-4ED7-90CB-6465D0729A7F},POINT (-75.742574 45.44649)
1291,81017,1711,Autres infrastructure,"9A, rue Front",Scène de l'Imaginaire,{F8981820-2D67-49D2-BC03-E8211FDD1919},POINT (-75.855441 45.395319)
1292,81017,1730,Monument et site historique,10 rue Bruyère,Monument en Mémoire-d'Elles,{F9EA7519-B3A4-48DB-AE6E-BDE0D61EE551},POINT (-75.700154 45.458813)
1293,81017,1741,Piscine,"120, rue Charlevoix",Piscine Fontaine,{2F5D6961-3FE7-4BB2-8B85-A2745209C81E},POINT (-75.716038 45.431642)


In [333]:
new_col_names = {
    "munid": "munid",
    "codeid": "codeid",
    "type": "type",
    "adr_comple": "lieu",
    "nom_topogr": "nom_topographique",
    "entiteid": "entiteid",
    "geom": "geom"
}

data2 = data2.rename(columns=new_col_names)

In [334]:
data2["type"].value_counts()

type
Parc récréatif                     208
Autre parc                         151
Service de garderie                132
École primaire                      67
Piscine                             63
                                  ... 
Aéroport                             1
Centre de détention                  1
Centre de conférence et congrès      1
Service électrique                   1
Centre militaire                     1
Name: count, Length: 67, dtype: int64

In [335]:
data2 = data2.copy()

data2[["longitude", "latitude"]] = data2["geom"].str.extract(
    r"POINT \((-?\d+\.\d+)\s+(-?\d+\.\d+)\)"
)

data2.loc[:, "longitude"] = data2["longitude"].astype(float)
data2.loc[:, "latitude"]  = data2["latitude"].astype(float)

In [336]:
data2.dtypes

munid                 int64
codeid                int64
type                 object
lieu                 object
nom_topographique    object
entiteid             object
geom                 object
longitude            object
latitude             object
dtype: object

In [337]:
data2.duplicated().sum()

0

In [338]:
data2.isna().sum()

munid                0
codeid               0
type                 0
lieu                 1
nom_topographique    0
entiteid             0
geom                 0
longitude            0
latitude             0
dtype: int64

In [339]:
data2.isnull().sum()

munid                0
codeid               0
type                 0
lieu                 1
nom_topographique    0
entiteid             0
geom                 0
longitude            0
latitude             0
dtype: int64

In [340]:
data2[data2["lieu"].isnull()]

,munid,codeid,type,lieu,nom_topographique,entiteid,geom,longitude,latitude
1131,81017,1569,Pont,NaN,Pont des Bénévoles,{7F9E3748-0619-4D54-A098-0C5B3E4675A2},POINT (-75.600554 45.503072),-75.600554,45.503072


In [341]:
data2[data2["type"] == "Pont"]

,munid,codeid,type,lieu,nom_topographique,entiteid,geom,longitude,latitude
53,81017,732,Pont,Rue Leclerc,Pont Brabant-Philippe (piste cyclable),{2F2B7A7D-E321-4835-B1D9-97D0AF1DD256},POINT (-75.594198 45.498357),-75.594198,45.498357
369,81017,37,Pont,1 Avenue du Pont,Pont Alonzo-Wright,{242472F7-EF10-45CD-A082-0DA9BAC4DA11},POINT (-75.748566 45.487222),-75.748566,45.487222
370,81017,65,Pont,2 Autoroute de la Gatineau,Pont Cartier-Macdonald,{FCBA028F-8907-4290-8931-A4497DEA9783},POINT (-75.703305 45.437093),-75.703305,45.437093
371,81017,164,Pont,2 Boulevard des Allumettières,Pont Alexandra,{AA083EDD-976D-4545-B435-0343300324BB},POINT (-75.704854 45.430333),-75.704854,45.430333
372,81017,76,Pont,2 Boulevard Maisonneuve,Pont du Portage,{95906E37-0657-449F-9549-4EEACC8BF56D},POINT (-75.713766 45.422688),-75.713766,45.422688
373,81017,141,Pont,2 Rue Eddy,Pont des Chaudières,{83CF259B-8861-4989-A606-3BF5BB8AF570},POINT (-75.718672 45.421678),-75.718672,45.421678
401,81017,142,Pont,2 Place Samuel-De Champlain,Pont Champlain,{7DDA212E-D730-48BE-B8A3-8E904792E5F1},POINT (-75.762259 45.411363),-75.762259,45.411363
402,81017,39,Pont,1500 Rue Saint-Louis,Pont des Draveurs,{32E7B57C-0799-4D36-B0BC-22FD842A646A},POINT (-75.716087 45.457147),-75.716087,45.457147
403,81017,55,Pont,1 Boulevard Gréber,Pont Lady-Aberdeen,{B716C93E-5572-44FD-9505-BB466ACED0F9},POINT (-75.701222 45.456858),-75.701222,45.456858
404,81017,608,Pont,1 Rue Maclaren Ouest,Pont Brady,{FC7BF868-45E4-46C8-9C16-F24BF24DBA06},POINT (-75.418751 45.585952),-75.418751,45.585952


### Recherche Google 

L'adresse correspondant aux coordonnées Latitude: 45.503072, Longitude: -75.600554 se situe dans le Parc de la Rivière Blanche, un espace naturel public à Gatineau, Québec. Ce parc est connu pour ses trois ponts couverts pittoresques, dont le Pont des Bénévoles, inauguré en 2014 .

📍 Localisation

Adresse la plus proche : Parc de la Rivière Blanche, Gatineau, Québec.

Secteur : Secteur Gatineau.
Wikipedia
+2
Centre d'action bénévole Accès
+2

Le parc est accessible via plusieurs sentiers pédestres. Le Pont des Bénévoles est l'un des trois ponts couverts du parc, offrant une expérience immersive en pleine nature.


In [342]:
data2["lieu"] = data2["lieu"].fillna("Parc de la Rivière Blanche")


In [343]:
data2[data2["type"] == "Pont"]

,munid,codeid,type,lieu,nom_topographique,entiteid,geom,longitude,latitude
53,81017,732,Pont,Rue Leclerc,Pont Brabant-Philippe (piste cyclable),{2F2B7A7D-E321-4835-B1D9-97D0AF1DD256},POINT (-75.594198 45.498357),-75.594198,45.498357
369,81017,37,Pont,1 Avenue du Pont,Pont Alonzo-Wright,{242472F7-EF10-45CD-A082-0DA9BAC4DA11},POINT (-75.748566 45.487222),-75.748566,45.487222
370,81017,65,Pont,2 Autoroute de la Gatineau,Pont Cartier-Macdonald,{FCBA028F-8907-4290-8931-A4497DEA9783},POINT (-75.703305 45.437093),-75.703305,45.437093
371,81017,164,Pont,2 Boulevard des Allumettières,Pont Alexandra,{AA083EDD-976D-4545-B435-0343300324BB},POINT (-75.704854 45.430333),-75.704854,45.430333
372,81017,76,Pont,2 Boulevard Maisonneuve,Pont du Portage,{95906E37-0657-449F-9549-4EEACC8BF56D},POINT (-75.713766 45.422688),-75.713766,45.422688
373,81017,141,Pont,2 Rue Eddy,Pont des Chaudières,{83CF259B-8861-4989-A606-3BF5BB8AF570},POINT (-75.718672 45.421678),-75.718672,45.421678
401,81017,142,Pont,2 Place Samuel-De Champlain,Pont Champlain,{7DDA212E-D730-48BE-B8A3-8E904792E5F1},POINT (-75.762259 45.411363),-75.762259,45.411363
402,81017,39,Pont,1500 Rue Saint-Louis,Pont des Draveurs,{32E7B57C-0799-4D36-B0BC-22FD842A646A},POINT (-75.716087 45.457147),-75.716087,45.457147
403,81017,55,Pont,1 Boulevard Gréber,Pont Lady-Aberdeen,{B716C93E-5572-44FD-9505-BB466ACED0F9},POINT (-75.701222 45.456858),-75.701222,45.456858
404,81017,608,Pont,1 Rue Maclaren Ouest,Pont Brady,{FC7BF868-45E4-46C8-9C16-F24BF24DBA06},POINT (-75.418751 45.585952),-75.418751,45.585952


In [344]:
data2.isna().sum()

munid                0
codeid               0
type                 0
lieu                 0
nom_topographique    0
entiteid             0
geom                 0
longitude            0
latitude             0
dtype: int64

In [345]:
data2.to_csv("dataset_cleaned/lieu_publique_clean.csv") 

### Cleaning Lieux publiques for Postgres

In [346]:
data_lieu = pd.read_csv("dataset_cleaned/lieu_publique_clean.csv")

In [347]:
data_lieu

,Unnamed: 0,munid,codeid,type,lieu,nom_topographique,entiteid,geom,longitude,latitude
0,0,81017,160,Édifice fédéral,625 Boulevard du Carrefour,Archives Nationales,{C22FB94A-24C3-4F58-8375-231468598ED2},POINT (-75.676786 45.487452),-75.676786,45.487452
1,1,81017,79,Résidence pour personnes âgées,60 Rue de la Futaie,Résidence Cité-Jardin,{88DE67A5-88F2-47D7-B4AA-BEDC8049F255},POINT (-75.68749 45.486385),-75.687490,45.486385
2,2,81017,570,Cinéma,120 Boulevard de l'Hôpital,Cinéma 9,{C1DB53E3-86E2-4BF5-ADE1-EEA62F3DB816},POINT (-75.686799 45.481),-75.686799,45.481000
3,3,81017,1162,Autres services médicaux et de santé,25 Rue Saint-François,Centre de réadaptation en dépendance,{C640A131-656C-4A21-AE93-D6120CDE0A0A},POINT (-75.748249 45.422431),-75.748249,45.422431
4,4,81017,1163,C.H.S.L.D.,134 Rue Jean-René-Monette,Centre d'hébergement Bon Séjour,{52C9F9B7-3721-4204-A13B-388828278AD5},POINT (-75.647589 45.481621),-75.647589,45.481621
...,...,...,...,...,...,...,...,...,...,...
1290,1290,81017,1698,Autres infrastructure,"97, boulevard Saint-Raymond",Planche à roulettes,{F815E3DF-1E6E-4ED7-90CB-6465D0729A7F},POINT (-75.742574 45.44649),-75.742574,45.446490
1291,1291,81017,1711,Autres infrastructure,"9A, rue Front",Scène de l'Imaginaire,{F8981820-2D67-49D2-BC03-E8211FDD1919},POINT (-75.855441 45.395319),-75.855441,45.395319
1292,1292,81017,1730,Monument et site historique,10 rue Bruyère,Monument en Mémoire-d'Elles,{F9EA7519-B3A4-48DB-AE6E-BDE0D61EE551},POINT (-75.700154 45.458813),-75.700154,45.458813
1293,1293,81017,1741,Piscine,"120, rue Charlevoix",Piscine Fontaine,{2F5D6961-3FE7-4BB2-8B85-A2745209C81E},POINT (-75.716038 45.431642),-75.716038,45.431642


In [348]:
data_lieu.drop(columns=["Unnamed: 0", "munid"], inplace=True)

In [349]:
data_lieu

,codeid,type,lieu,nom_topographique,entiteid,geom,longitude,latitude
0,160,Édifice fédéral,625 Boulevard du Carrefour,Archives Nationales,{C22FB94A-24C3-4F58-8375-231468598ED2},POINT (-75.676786 45.487452),-75.676786,45.487452
1,79,Résidence pour personnes âgées,60 Rue de la Futaie,Résidence Cité-Jardin,{88DE67A5-88F2-47D7-B4AA-BEDC8049F255},POINT (-75.68749 45.486385),-75.687490,45.486385
2,570,Cinéma,120 Boulevard de l'Hôpital,Cinéma 9,{C1DB53E3-86E2-4BF5-ADE1-EEA62F3DB816},POINT (-75.686799 45.481),-75.686799,45.481000
3,1162,Autres services médicaux et de santé,25 Rue Saint-François,Centre de réadaptation en dépendance,{C640A131-656C-4A21-AE93-D6120CDE0A0A},POINT (-75.748249 45.422431),-75.748249,45.422431
4,1163,C.H.S.L.D.,134 Rue Jean-René-Monette,Centre d'hébergement Bon Séjour,{52C9F9B7-3721-4204-A13B-388828278AD5},POINT (-75.647589 45.481621),-75.647589,45.481621
...,...,...,...,...,...,...,...,...
1290,1698,Autres infrastructure,"97, boulevard Saint-Raymond",Planche à roulettes,{F815E3DF-1E6E-4ED7-90CB-6465D0729A7F},POINT (-75.742574 45.44649),-75.742574,45.446490
1291,1711,Autres infrastructure,"9A, rue Front",Scène de l'Imaginaire,{F8981820-2D67-49D2-BC03-E8211FDD1919},POINT (-75.855441 45.395319),-75.855441,45.395319
1292,1730,Monument et site historique,10 rue Bruyère,Monument en Mémoire-d'Elles,{F9EA7519-B3A4-48DB-AE6E-BDE0D61EE551},POINT (-75.700154 45.458813),-75.700154,45.458813
1293,1741,Piscine,"120, rue Charlevoix",Piscine Fontaine,{2F5D6961-3FE7-4BB2-8B85-A2745209C81E},POINT (-75.716038 45.431642),-75.716038,45.431642


In [350]:
data_lieu.duplicated().sum()

0

In [351]:
data_lieu.isna().sum()

codeid               0
type                 0
lieu                 0
nom_topographique    0
entiteid             0
geom                 0
longitude            0
latitude             0
dtype: int64

In [352]:
data_lieu.rename(columns={"codeid": "lieuid"}, inplace=True)

In [353]:
data_lieu

,lieuid,type,lieu,nom_topographique,entiteid,geom,longitude,latitude
0,160,Édifice fédéral,625 Boulevard du Carrefour,Archives Nationales,{C22FB94A-24C3-4F58-8375-231468598ED2},POINT (-75.676786 45.487452),-75.676786,45.487452
1,79,Résidence pour personnes âgées,60 Rue de la Futaie,Résidence Cité-Jardin,{88DE67A5-88F2-47D7-B4AA-BEDC8049F255},POINT (-75.68749 45.486385),-75.687490,45.486385
2,570,Cinéma,120 Boulevard de l'Hôpital,Cinéma 9,{C1DB53E3-86E2-4BF5-ADE1-EEA62F3DB816},POINT (-75.686799 45.481),-75.686799,45.481000
3,1162,Autres services médicaux et de santé,25 Rue Saint-François,Centre de réadaptation en dépendance,{C640A131-656C-4A21-AE93-D6120CDE0A0A},POINT (-75.748249 45.422431),-75.748249,45.422431
4,1163,C.H.S.L.D.,134 Rue Jean-René-Monette,Centre d'hébergement Bon Séjour,{52C9F9B7-3721-4204-A13B-388828278AD5},POINT (-75.647589 45.481621),-75.647589,45.481621
...,...,...,...,...,...,...,...,...
1290,1698,Autres infrastructure,"97, boulevard Saint-Raymond",Planche à roulettes,{F815E3DF-1E6E-4ED7-90CB-6465D0729A7F},POINT (-75.742574 45.44649),-75.742574,45.446490
1291,1711,Autres infrastructure,"9A, rue Front",Scène de l'Imaginaire,{F8981820-2D67-49D2-BC03-E8211FDD1919},POINT (-75.855441 45.395319),-75.855441,45.395319
1292,1730,Monument et site historique,10 rue Bruyère,Monument en Mémoire-d'Elles,{F9EA7519-B3A4-48DB-AE6E-BDE0D61EE551},POINT (-75.700154 45.458813),-75.700154,45.458813
1293,1741,Piscine,"120, rue Charlevoix",Piscine Fontaine,{2F5D6961-3FE7-4BB2-8B85-A2745209C81E},POINT (-75.716038 45.431642),-75.716038,45.431642


In [354]:
url = "https://www.gatineau.ca/upload/donneesouvertes/DECOUPAGE_ADMINISTRATIF.json" 
gdf = gpd.read_file(url)
print(gdf.head())


   MUNID CODEID            NOM      TYPE ABREV          AIRE     PERIM  \
0  81017      5     Buckingham  Ex ville     B  1.608119e+07  18214.35   
1  81017     10  Masson-Angers  Ex ville     M  6.453902e+07  46386.20   
2  81017     15       Gatineau  Ex ville     G  1.585884e+08  58086.51   
3  81017     20           Hull  Ex ville     H  4.133173e+07  32973.84   
4  81017     25         Aylmer  Ex ville     A  1.000886e+08  42578.92   

                                 ENTITEID  \
0  {0DE4611D-2A9E-4B26-A1B6-72A9C3DD3DBD}   
1  {96AAFDBC-9949-41EF-8D00-ECE15E302886}   
2  {07DBC407-0068-4AEE-8F5E-51A985940380}   
3  {36908290-4C89-41DB-A00A-C29EB4CFB9A8}   
4  {5C1FA124-6734-44C6-AB31-23B28DC0EF91}   

                                            geometry  
0  POLYGON ((-75.41524 45.59898, -75.41522 45.598...  
1  POLYGON ((-75.35372 45.5957, -75.35272 45.5878...  
2  POLYGON ((-75.53744 45.57731, -75.5331 45.5479...  
3  POLYGON ((-75.77524 45.48664, -75.7751 45.4862...  
4  POLYGO

In [355]:
# Transformer en GeoDataFrame
geometry = [Point(xy) for xy in zip(data_lieu["longitude"], data_lieu["latitude"])]
gdf_points = gpd.GeoDataFrame(data_lieu, geometry=geometry, crs="EPSG:4326")


In [356]:
gdf = gdf.to_crs("EPSG:4326")

In [357]:
geo2 = gpd.sjoin(gdf_points, gdf, how="left", predicate="within")

In [358]:
geo2.head()

,lieuid,type,lieu,nom_topographique,entiteid,geom,longitude,latitude,geometry,index_right,MUNID,CODEID,NOM,TYPE,ABREV,AIRE,PERIM,ENTITEID
0,160,Édifice fédéral,625 Boulevard du Carrefour,Archives Nationales,{C22FB94A-24C3-4F58-8375-231468598ED2},POINT (-75.676786 45.487452),-75.676786,45.487452,POINT (-75.67679 45.48745),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
1,79,Résidence pour personnes âgées,60 Rue de la Futaie,Résidence Cité-Jardin,{88DE67A5-88F2-47D7-B4AA-BEDC8049F255},POINT (-75.68749 45.486385),-75.687490,45.486385,POINT (-75.68749 45.48638),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
2,570,Cinéma,120 Boulevard de l'Hôpital,Cinéma 9,{C1DB53E3-86E2-4BF5-ADE1-EEA62F3DB816},POINT (-75.686799 45.481),-75.686799,45.481000,POINT (-75.6868 45.481),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
3,1162,Autres services médicaux et de santé,25 Rue Saint-François,Centre de réadaptation en dépendance,{C640A131-656C-4A21-AE93-D6120CDE0A0A},POINT (-75.748249 45.422431),-75.748249,45.422431,POINT (-75.74825 45.42243),3.0,81017.0,20,Hull,Ex ville,H,4.133173e+07,32973.84,{36908290-4C89-41DB-A00A-C29EB4CFB9A8}
4,1163,C.H.S.L.D.,134 Rue Jean-René-Monette,Centre d'hébergement Bon Séjour,{52C9F9B7-3721-4204-A13B-388828278AD5},POINT (-75.647589 45.481621),-75.647589,45.481621,POINT (-75.64759 45.48162),2.0,81017.0,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}


In [359]:
geo2.columns

Index(['lieuid', 'type', 'lieu', 'nom_topographique', 'entiteid', 'geom',
       'longitude', 'latitude', 'geometry', 'index_right', 'MUNID', 'CODEID',
       'NOM', 'TYPE', 'ABREV', 'AIRE', 'PERIM', 'ENTITEID'],
      dtype='object')

In [360]:
geo2 = geo2.drop(columns=[
    'geometry',
    'index_right',
    'MUNID',
    'NOM',
    'TYPE',
    'ABREV',
    'AIRE',
    'PERIM',
    'ENTITEID'
])

In [361]:
geo2.columns

Index(['lieuid', 'type', 'lieu', 'nom_topographique', 'entiteid', 'geom',
       'longitude', 'latitude', 'CODEID'],
      dtype='object')

In [362]:
geo2_dict = {
    "lieuid": "lieu_publique_id",
    "type": "type",
    "lieu": "lieu",
    'nom_topographique': 'nom_topographique',
    "entiteid": "entiteid",
    "geom": "geom",
    "longitude": "longitude",
    "latitude": "latitude",
    "CODEID": "decoupageid"
}

geo2 = geo2.rename(columns=geo2_dict)

In [363]:
geo2

,lieu_publique_id,type,lieu,nom_topographique,entiteid,geom,longitude,latitude,decoupageid
0,160,Édifice fédéral,625 Boulevard du Carrefour,Archives Nationales,{C22FB94A-24C3-4F58-8375-231468598ED2},POINT (-75.676786 45.487452),-75.676786,45.487452,15
1,79,Résidence pour personnes âgées,60 Rue de la Futaie,Résidence Cité-Jardin,{88DE67A5-88F2-47D7-B4AA-BEDC8049F255},POINT (-75.68749 45.486385),-75.687490,45.486385,15
2,570,Cinéma,120 Boulevard de l'Hôpital,Cinéma 9,{C1DB53E3-86E2-4BF5-ADE1-EEA62F3DB816},POINT (-75.686799 45.481),-75.686799,45.481000,15
3,1162,Autres services médicaux et de santé,25 Rue Saint-François,Centre de réadaptation en dépendance,{C640A131-656C-4A21-AE93-D6120CDE0A0A},POINT (-75.748249 45.422431),-75.748249,45.422431,20
4,1163,C.H.S.L.D.,134 Rue Jean-René-Monette,Centre d'hébergement Bon Séjour,{52C9F9B7-3721-4204-A13B-388828278AD5},POINT (-75.647589 45.481621),-75.647589,45.481621,15
...,...,...,...,...,...,...,...,...,...
1290,1698,Autres infrastructure,"97, boulevard Saint-Raymond",Planche à roulettes,{F815E3DF-1E6E-4ED7-90CB-6465D0729A7F},POINT (-75.742574 45.44649),-75.742574,45.446490,20
1291,1711,Autres infrastructure,"9A, rue Front",Scène de l'Imaginaire,{F8981820-2D67-49D2-BC03-E8211FDD1919},POINT (-75.855441 45.395319),-75.855441,45.395319,25
1292,1730,Monument et site historique,10 rue Bruyère,Monument en Mémoire-d'Elles,{F9EA7519-B3A4-48DB-AE6E-BDE0D61EE551},POINT (-75.700154 45.458813),-75.700154,45.458813,15
1293,1741,Piscine,"120, rue Charlevoix",Piscine Fontaine,{2F5D6961-3FE7-4BB2-8B85-A2745209C81E},POINT (-75.716038 45.431642),-75.716038,45.431642,20


In [364]:
geo2[geo2["lieu_publique_id"].duplicated(keep=False)].sort_values("lieu_publique_id")


,lieu_publique_id,type,lieu,nom_topographique,entiteid,geom,longitude,latitude,decoupageid
738,666,Autre parc,205 Rue de Bruxelles,Parc Central,{E7690AA3-5842-4CCA-869C-E2EBFD3B8F7F},POINT (-75.80139 45.433626),-75.801390,45.433626,25
1058,666,Piscine,205 Rue de Bruxelles,Jeux d'eau Central,{CB108143-D5F9-442D-BE96-7D2D48A7CA0B},POINT (-75.80098 45.433958),-75.800980,45.433958,25
556,1149,Service de garderie,355 Boulevard de la Cité-des-Jeunes,CPE Coop Libelle,{4BF9DFED-D221-4813-BD14-78201235E4E6},POINT (-75.768661 45.460813),-75.768661,45.460813,20
1157,1149,Service de garderie,365 Boulevard de la Cité-des-Jeunes,CPE Coop Libelle 2,{B0E2C14A-7802-4DFB-9374-08EECC6B249D},POINT (-75.769065 45.461112),-75.769065,45.461112,20


In [365]:
geo2['lieuid'] = range(1, len(geo2) + 1)
cols = list(geo2.columns)
cols.remove('lieuid')
cols.insert(0, 'lieuid')

# 3. Appliquer le nouvel ordre au DataFrame
geo2 = geo2[cols]

In [366]:
geo2

,lieuid,lieu_publique_id,type,lieu,nom_topographique,entiteid,geom,longitude,latitude,decoupageid
0,1,160,Édifice fédéral,625 Boulevard du Carrefour,Archives Nationales,{C22FB94A-24C3-4F58-8375-231468598ED2},POINT (-75.676786 45.487452),-75.676786,45.487452,15
1,2,79,Résidence pour personnes âgées,60 Rue de la Futaie,Résidence Cité-Jardin,{88DE67A5-88F2-47D7-B4AA-BEDC8049F255},POINT (-75.68749 45.486385),-75.687490,45.486385,15
2,3,570,Cinéma,120 Boulevard de l'Hôpital,Cinéma 9,{C1DB53E3-86E2-4BF5-ADE1-EEA62F3DB816},POINT (-75.686799 45.481),-75.686799,45.481000,15
3,4,1162,Autres services médicaux et de santé,25 Rue Saint-François,Centre de réadaptation en dépendance,{C640A131-656C-4A21-AE93-D6120CDE0A0A},POINT (-75.748249 45.422431),-75.748249,45.422431,20
4,5,1163,C.H.S.L.D.,134 Rue Jean-René-Monette,Centre d'hébergement Bon Séjour,{52C9F9B7-3721-4204-A13B-388828278AD5},POINT (-75.647589 45.481621),-75.647589,45.481621,15
...,...,...,...,...,...,...,...,...,...,...
1290,1291,1698,Autres infrastructure,"97, boulevard Saint-Raymond",Planche à roulettes,{F815E3DF-1E6E-4ED7-90CB-6465D0729A7F},POINT (-75.742574 45.44649),-75.742574,45.446490,20
1291,1292,1711,Autres infrastructure,"9A, rue Front",Scène de l'Imaginaire,{F8981820-2D67-49D2-BC03-E8211FDD1919},POINT (-75.855441 45.395319),-75.855441,45.395319,25
1292,1293,1730,Monument et site historique,10 rue Bruyère,Monument en Mémoire-d'Elles,{F9EA7519-B3A4-48DB-AE6E-BDE0D61EE551},POINT (-75.700154 45.458813),-75.700154,45.458813,15
1293,1294,1741,Piscine,"120, rue Charlevoix",Piscine Fontaine,{2F5D6961-3FE7-4BB2-8B85-A2745209C81E},POINT (-75.716038 45.431642),-75.716038,45.431642,20


In [367]:
geo2.isnull().sum()

lieuid               0
lieu_publique_id     0
type                 0
lieu                 0
nom_topographique    0
entiteid             0
geom                 0
longitude            0
latitude             0
decoupageid          1
dtype: int64

In [368]:
geo2[geo2['decoupageid'].isnull()]


,lieuid,lieu_publique_id,type,lieu,nom_topographique,entiteid,geom,longitude,latitude,decoupageid
528,529,838,Rivière et ruisseau,,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,NaN


In [369]:
geo2.loc[
    (geo2['type'] == 'Rivière et ruisseau') & (geo2['lieu'].str.strip() == ''), 
    'lieu'
] = geo2['nom_topographique']

In [370]:
geo2[geo2['decoupageid'].isnull()]

,lieuid,lieu_publique_id,type,lieu,nom_topographique,entiteid,geom,longitude,latitude,decoupageid
528,529,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,NaN


In [371]:
rivière = geo2[geo2['nom_topographique'] == 'Rivière des Outaouais']
secteurs_ids = [25, 20, 15, 10, 5]  # Aylmer, Hull, Gatineau, Masson-Angers, Buckingham
# Réplication
rivière_multi = pd.concat([
    rivière.assign(decoupageid=secteur_id) for secteur_id in secteurs_ids
], ignore_index=True)
geo2 = pd.concat([geo2, rivière_multi], ignore_index=True)

# Réassigner lieuid unique sur tout le DataFrame
geo2['lieuid'] = range(1, len(geo2) + 1)


In [372]:
#geo2 = geo2[~((geo2['nom_topographique'] == 'Rivière des Outaouais') & (geo2['decoupageid'].isnull()))]


In [373]:
rivière = geo2[geo2['nom_topographique'] == 'Rivière des Outaouais']
rivière

,lieuid,lieu_publique_id,type,lieu,nom_topographique,entiteid,geom,longitude,latitude,decoupageid
528,529,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,NaN
1295,1296,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,25
1296,1297,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,20
1297,1298,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,15
1298,1299,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,10
1299,1300,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,5


In [374]:
geo2 = geo2[geo2['decoupageid'].notnull()].reset_index(drop=True)

In [375]:
rivière = geo2[geo2['nom_topographique'] == 'Rivière des Outaouais']
rivière

,lieuid,lieu_publique_id,type,lieu,nom_topographique,entiteid,geom,longitude,latitude,decoupageid
1294,1296,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,25
1295,1297,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,20
1296,1298,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,15
1297,1299,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,10
1298,1300,838,Rivière et ruisseau,Rivière des Outaouais,Rivière des Outaouais,{671E08BD-FB5F-4A68-92E5-44C3085BC023},POINT (-75.773275 45.392076),-75.773275,45.392076,5


In [376]:
# Trouver toutes les lignes où 'lieuid' est dupliqué (toutes occurrences)
doublons = geo2[geo2['lieuid'].duplicated(keep=False)]

# Afficher ces lignes
print(doublons)

# Optionnel : juste savoir si des doublons existent
if doublons.empty:
    print("Pas de doublons dans la colonne 'lieuid'.")
else:
    print(f"Il y a {doublons['lieuid'].nunique()} valeurs dupliquées dans 'lieuid'.")


Empty DataFrame
Columns: [lieuid, lieu_publique_id, type, lieu, nom_topographique, entiteid, geom, longitude, latitude, decoupageid]
Index: []
Pas de doublons dans la colonne 'lieuid'.


In [377]:
geo2.dtypes

lieuid                 int64
lieu_publique_id       int64
type                  object
lieu                  object
nom_topographique     object
entiteid              object
geom                  object
longitude            float64
latitude             float64
decoupageid           object
dtype: object

In [378]:
geo2.dtypes

lieuid                 int64
lieu_publique_id       int64
type                  object
lieu                  object
nom_topographique     object
entiteid              object
geom                  object
longitude            float64
latitude             float64
decoupageid           object
dtype: object

In [379]:
dupes = geo2[geo2.duplicated(subset=['lieu_publique_id'], keep=False)]
print(dupes)
print(f"Nombre de doublons : {dupes.shape[0]}")


      lieuid  lieu_publique_id                 type  \
555      557              1149  Service de garderie   
737      739               666           Autre parc   
1057    1059               666              Piscine   
1156    1158              1149  Service de garderie   
1294    1296               838  Rivière et ruisseau   
1295    1297               838  Rivière et ruisseau   
1296    1298               838  Rivière et ruisseau   
1297    1299               838  Rivière et ruisseau   
1298    1300               838  Rivière et ruisseau   

                                     lieu      nom_topographique  \
555   355 Boulevard de la Cité-des-Jeunes      CPE Coop Libelle    
737                  205 Rue de Bruxelles           Parc Central   
1057                 205 Rue de Bruxelles     Jeux d'eau Central   
1156  365 Boulevard de la Cité-des-Jeunes     CPE Coop Libelle 2   
1294                Rivière des Outaouais  Rivière des Outaouais   
1295                Rivière des Outaouais

In [380]:
geo2.head(50)

,lieuid,lieu_publique_id,type,lieu,nom_topographique,entiteid,geom,longitude,latitude,decoupageid
0,1,160,Édifice fédéral,625 Boulevard du Carrefour,Archives Nationales,{C22FB94A-24C3-4F58-8375-231468598ED2},POINT (-75.676786 45.487452),-75.676786,45.487452,15
1,2,79,Résidence pour personnes âgées,60 Rue de la Futaie,Résidence Cité-Jardin,{88DE67A5-88F2-47D7-B4AA-BEDC8049F255},POINT (-75.68749 45.486385),-75.687490,45.486385,15
2,3,570,Cinéma,120 Boulevard de l'Hôpital,Cinéma 9,{C1DB53E3-86E2-4BF5-ADE1-EEA62F3DB816},POINT (-75.686799 45.481),-75.686799,45.481000,15
3,4,1162,Autres services médicaux et de santé,25 Rue Saint-François,Centre de réadaptation en dépendance,{C640A131-656C-4A21-AE93-D6120CDE0A0A},POINT (-75.748249 45.422431),-75.748249,45.422431,20
4,5,1163,C.H.S.L.D.,134 Rue Jean-René-Monette,Centre d'hébergement Bon Séjour,{52C9F9B7-3721-4204-A13B-388828278AD5},POINT (-75.647589 45.481621),-75.647589,45.481621,15
5,6,1166,Usine de traitement des eaux,291 Boulevard de Lucerne,Usine de filtration Secteur Hull,{AC16966C-8B44-42D1-B169-453A3D395524},POINT (-75.753116 45.419376),-75.753116,45.419376,20
6,7,69,Édifice municipal,170 Rue de l'Hôtel-de-Ville,Édifice Jos-Monferrand,{2B57949C-E3F6-42FC-8F71-B032B39745E0},POINT (-75.711457 45.426872),-75.711457,45.426872,20
7,8,171,Centre communautaire,70 Rue Moreau,Centre communautaire Saint-Jean-De Brébeuf,{7BBAFC5F-1474-42DB-9D88-EB20E0278B69},POINT (-75.707064 45.46431),-75.707064,45.464310,15
8,9,441,Parc récréatif,520 Rue Carle,Parc Apollo,{41A4D5D5-1B2B-41DF-8E4B-9BD7A5505AF0},POINT (-75.670275 45.487214),-75.670275,45.487214,15
9,10,146,Centre sportif,211 Rue Montcalm,La Fonderie,{5AFF6B0A-3425-4CEF-8C21-F588ACAD982D},POINT (-75.731136 45.430451),-75.731136,45.430451,20


In [381]:
geo2['decoupageid'].isna().sum()

0

In [382]:
# Afficher les lignes avec decoupageid null / NaN
lignes_null = geo2[geo2['decoupageid'].isnull()]

print(lignes_null)


Empty DataFrame
Columns: [lieuid, lieu_publique_id, type, lieu, nom_topographique, entiteid, geom, longitude, latitude, decoupageid]
Index: []


In [383]:
geo2.to_csv("pg_cleaned/lieu_geo.csv", index= False)

### Decoupage Administratif

In [384]:
data3 = pd.read_csv("raw_data/decoupage_administratif.csv")

In [385]:
data3

,MUNID,CODEID,NOM,TYPE,ABREV,AIRE,PERIM,ENTITEID,GEOM
0,81017,5,Buckingham,Ex ville,B,1.608119e+07,18214.35,{0DE4611D-2A9E-4B26-A1B6-72A9C3DD3DBD},"POLYGON (-75.415244 45.598979,-75.41522 45.598..."
1,81017,10,Masson-Angers,Ex ville,M,6.453902e+07,46386.20,{96AAFDBC-9949-41EF-8D00-ECE15E302886},"POLYGON (-75.353715 45.595704,-75.352718 45.58..."
2,81017,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380},"POLYGON (-75.53744 45.577313,-75.533105 45.547..."
3,81017,20,Hull,Ex ville,H,4.133173e+07,32973.84,{36908290-4C89-41DB-A00A-C29EB4CFB9A8},"POLYGON (-75.775238 45.486638,-75.775096 45.48..."
4,81017,25,Aylmer,Ex ville,A,1.000886e+08,42578.92,{5C1FA124-6734-44C6-AB31-23B28DC0EF91},"POLYGON (-75.83943 45.485675,-75.837062 45.470..."


In [386]:
data3.columns = data3.columns.str.lower()

In [387]:
data3.columns

Index(['munid', 'codeid', 'nom', 'type', 'abrev', 'aire', 'perim', 'entiteid',
       'geom'],
      dtype='object')

In [388]:
data3.head()

,munid,codeid,nom,type,abrev,aire,perim,entiteid,geom
0,81017,5,Buckingham,Ex ville,B,1.608119e+07,18214.35,{0DE4611D-2A9E-4B26-A1B6-72A9C3DD3DBD},"POLYGON (-75.415244 45.598979,-75.41522 45.598..."
1,81017,10,Masson-Angers,Ex ville,M,6.453902e+07,46386.20,{96AAFDBC-9949-41EF-8D00-ECE15E302886},"POLYGON (-75.353715 45.595704,-75.352718 45.58..."
2,81017,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380},"POLYGON (-75.53744 45.577313,-75.533105 45.547..."
3,81017,20,Hull,Ex ville,H,4.133173e+07,32973.84,{36908290-4C89-41DB-A00A-C29EB4CFB9A8},"POLYGON (-75.775238 45.486638,-75.775096 45.48..."
4,81017,25,Aylmer,Ex ville,A,1.000886e+08,42578.92,{5C1FA124-6734-44C6-AB31-23B28DC0EF91},"POLYGON (-75.83943 45.485675,-75.837062 45.470..."


In [389]:
colonnes_dec_mapping = {
    "munid": "munid",
    "codeid": "codeid",
    "nom": "secteur",
    "type": "type",
    "abrev": "abreviation",
    "aire":"aire",
    "perim": "perimetre",   
    "entiteid": "entiteid",
    "geom": "geom",
   
}
data3= data3.rename(columns=colonnes_dec_mapping)
data3.columns


Index(['munid', 'codeid', 'secteur', 'type', 'abreviation', 'aire',
       'perimetre', 'entiteid', 'geom'],
      dtype='object')

In [390]:
data3.to_csv("dataset_cleaned/decoupage_administratif_clean.csv") 

In [391]:
data3.dtypes

munid            int64
codeid           int64
secteur         object
type            object
abreviation     object
aire           float64
perimetre      float64
entiteid        object
geom            object
dtype: object

### Cleaning Decoupage administratif for Postgres

In [392]:
data_decoupage = pd.read_csv("dataset_cleaned/decoupage_administratif_clean.csv")

In [393]:
data_decoupage

,Unnamed: 0,munid,codeid,secteur,type,abreviation,aire,perimetre,entiteid,geom
0,0,81017,5,Buckingham,Ex ville,B,1.608119e+07,18214.35,{0DE4611D-2A9E-4B26-A1B6-72A9C3DD3DBD},"POLYGON (-75.415244 45.598979,-75.41522 45.598..."
1,1,81017,10,Masson-Angers,Ex ville,M,6.453902e+07,46386.20,{96AAFDBC-9949-41EF-8D00-ECE15E302886},"POLYGON (-75.353715 45.595704,-75.352718 45.58..."
2,2,81017,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380},"POLYGON (-75.53744 45.577313,-75.533105 45.547..."
3,3,81017,20,Hull,Ex ville,H,4.133173e+07,32973.84,{36908290-4C89-41DB-A00A-C29EB4CFB9A8},"POLYGON (-75.775238 45.486638,-75.775096 45.48..."
4,4,81017,25,Aylmer,Ex ville,A,1.000886e+08,42578.92,{5C1FA124-6734-44C6-AB31-23B28DC0EF91},"POLYGON (-75.83943 45.485675,-75.837062 45.470..."


In [394]:
data_decoupage.columns

Index(['Unnamed: 0', 'munid', 'codeid', 'secteur', 'type', 'abreviation',
       'aire', 'perimetre', 'entiteid', 'geom'],
      dtype='object')

In [395]:
data_decoupage = data_decoupage.drop(columns=["Unnamed: 0","munid"])

In [396]:
data_decoupage.rename(columns={"codeid": "decoupageid"}, inplace=True)

In [397]:
data_decoupage

,decoupageid,secteur,type,abreviation,aire,perimetre,entiteid,geom
0,5,Buckingham,Ex ville,B,1.608119e+07,18214.35,{0DE4611D-2A9E-4B26-A1B6-72A9C3DD3DBD},"POLYGON (-75.415244 45.598979,-75.41522 45.598..."
1,10,Masson-Angers,Ex ville,M,6.453902e+07,46386.20,{96AAFDBC-9949-41EF-8D00-ECE15E302886},"POLYGON (-75.353715 45.595704,-75.352718 45.58..."
2,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380},"POLYGON (-75.53744 45.577313,-75.533105 45.547..."
3,20,Hull,Ex ville,H,4.133173e+07,32973.84,{36908290-4C89-41DB-A00A-C29EB4CFB9A8},"POLYGON (-75.775238 45.486638,-75.775096 45.48..."
4,25,Aylmer,Ex ville,A,1.000886e+08,42578.92,{5C1FA124-6734-44C6-AB31-23B28DC0EF91},"POLYGON (-75.83943 45.485675,-75.837062 45.470..."


In [398]:
data_decoupage.dtypes

decoupageid      int64
secteur         object
type            object
abreviation     object
aire           float64
perimetre      float64
entiteid        object
geom            object
dtype: object

In [399]:
data_decoupage.to_csv("pg_cleaned/decoupage_geo.csv", index=False)


### Adresse d'immeuble

In [400]:
data4 = pd.read_csv("raw_data/adresse_immeuble.csv")

In [401]:
data4

,CODEID,MUNID,NUMERO_CIV,GENERIQUE,LIAISON,SPECIFIQUE,DIRECTION,ADR_COMPLE,RUESID,ENTITEID,GEOM
0,197,81017,88,Rue,de,Lacaune,NaN,88 Rue de Lacaune,2638,{DA0C3CB2-7B9F-4891-ABFB-8A35E152F59F},POINT (-75.720493 45.489893)
1,198,81017,115,Rue,de,Lacaune,NaN,115 Rue de Lacaune,2638,{40FC4580-A84A-4E0C-9EB2-FC02EC3475C8},POINT (-75.721708 45.490631)
2,199,81017,112,Rue,de,Lacaune,NaN,112 Rue de Lacaune,2638,{B2DE71BD-D4F6-4014-BC10-B34768C3CCF5},POINT (-75.721607 45.490147)
3,200,81017,111,Rue,de,Lacaune,NaN,111 Rue de Lacaune,2638,{AF6656E2-C657-4F1E-9685-95A8B389C8D9},POINT (-75.721519 45.490596)
4,282,81017,87,Rue,de,Lacaune,NaN,87 Rue de Lacaune,2638,{A0FBFDA5-B98B-4AAB-ACFA-92384FFB4640},POINT (-75.720391 45.490386)
...,...,...,...,...,...,...,...,...,...,...,...
96007,99811,81017,46,Rue,NaN,Latour,NaN,46 Rue Latour,2673,{8C808A0D-6059-474C-ADAE-3DC80BC3844B},POINT (-75.399655 45.58534)
96008,99810,81017,44,Rue,NaN,Latour,NaN,44 Rue Latour,2673,{AC8A472D-DE7F-4C8C-8FC6-280B3D0D9F73},POINT (-75.399667 45.585397)
96009,99814,81017,52,Rue,NaN,Latour,NaN,52 Rue Latour,2673,{DC16CC64-15D7-4B41-B2F9-433913202838},POINT (-75.399556 45.585349)
96010,99819,81017,779(2),Boulevard,NaN,Maloney,Est,779(2) Boulevard Maloney Est,2147,{3E4B7F64-383A-4764-80FA-120FBE3DA902},POINT (-75.615936 45.483131)


In [402]:
data4.value_counts("DIRECTION")

DIRECTION
Est      1849
Ouest    1723
Nord      151
Sud       121
Name: count, dtype: int64

In [403]:
data4.duplicated().sum()

0

In [404]:
data4.columns = data4.columns.str.lower()

In [405]:
data4.columns

Index(['codeid', 'munid', 'numero_civ', 'generique', 'liaison', 'specifique',
       'direction', 'adr_comple', 'ruesid', 'entiteid', 'geom'],
      dtype='object')

In [406]:
data4["direction"] = data4["direction"].fillna("Non Attribué")

In [407]:
data4["liaison"] = data4["liaison"].fillna(" ")

In [408]:
data4["lieu"] = data4[["generique", "liaison", "specifique"]].astype(str).agg(" ".join, axis=1)

In [409]:
data4["lieu"] = data4["lieu"].str.replace(r"\s+", " ", regex=True).str.strip()

In [410]:
data4 = data4.drop(columns=["generique","liaison","specifique"])

In [411]:
data4 = data4[["codeid", "munid","numero_civ","lieu", "direction", "adr_comple", "ruesid", "entiteid", "geom"]]

In [412]:
data4

,codeid,munid,numero_civ,lieu,direction,adr_comple,ruesid,entiteid,geom
0,197,81017,88,Rue de Lacaune,Non Attribué,88 Rue de Lacaune,2638,{DA0C3CB2-7B9F-4891-ABFB-8A35E152F59F},POINT (-75.720493 45.489893)
1,198,81017,115,Rue de Lacaune,Non Attribué,115 Rue de Lacaune,2638,{40FC4580-A84A-4E0C-9EB2-FC02EC3475C8},POINT (-75.721708 45.490631)
2,199,81017,112,Rue de Lacaune,Non Attribué,112 Rue de Lacaune,2638,{B2DE71BD-D4F6-4014-BC10-B34768C3CCF5},POINT (-75.721607 45.490147)
3,200,81017,111,Rue de Lacaune,Non Attribué,111 Rue de Lacaune,2638,{AF6656E2-C657-4F1E-9685-95A8B389C8D9},POINT (-75.721519 45.490596)
4,282,81017,87,Rue de Lacaune,Non Attribué,87 Rue de Lacaune,2638,{A0FBFDA5-B98B-4AAB-ACFA-92384FFB4640},POINT (-75.720391 45.490386)
...,...,...,...,...,...,...,...,...,...
96007,99811,81017,46,Rue Latour,Non Attribué,46 Rue Latour,2673,{8C808A0D-6059-474C-ADAE-3DC80BC3844B},POINT (-75.399655 45.58534)
96008,99810,81017,44,Rue Latour,Non Attribué,44 Rue Latour,2673,{AC8A472D-DE7F-4C8C-8FC6-280B3D0D9F73},POINT (-75.399667 45.585397)
96009,99814,81017,52,Rue Latour,Non Attribué,52 Rue Latour,2673,{DC16CC64-15D7-4B41-B2F9-433913202838},POINT (-75.399556 45.585349)
96010,99819,81017,779(2),Boulevard Maloney,Est,779(2) Boulevard Maloney Est,2147,{3E4B7F64-383A-4764-80FA-120FBE3DA902},POINT (-75.615936 45.483131)


In [413]:
data4.duplicated().sum()

0

In [414]:
data4["adr_comple"].duplicated().sum()

0

In [415]:
data4.isnull().sum()

codeid        0
munid         0
numero_civ    0
lieu          0
direction     0
adr_comple    0
ruesid        0
entiteid      0
geom          0
dtype: int64

In [416]:
data4.isna().sum()

codeid        0
munid         0
numero_civ    0
lieu          0
direction     0
adr_comple    0
ruesid        0
entiteid      0
geom          0
dtype: int64

In [417]:
data4.duplicated().sum()  

0

In [418]:
colonnes_add_mapping = {
    "codeid": "codeid",
    "munid": "munid",
    "numero_civ": "numero_civ",
    "lieu": "lieu",
    "direction": "direction",
    "adr_comple": "adresse_complete",
    "ruesid": "ruesid",
    "entiteid": "entiteid",
    "geom": "geom",
}

# Appliquer le renommage
data4 = data4.rename(columns=colonnes_add_mapping)

In [419]:
data4

,codeid,munid,numero_civ,lieu,direction,adresse_complete,ruesid,entiteid,geom
0,197,81017,88,Rue de Lacaune,Non Attribué,88 Rue de Lacaune,2638,{DA0C3CB2-7B9F-4891-ABFB-8A35E152F59F},POINT (-75.720493 45.489893)
1,198,81017,115,Rue de Lacaune,Non Attribué,115 Rue de Lacaune,2638,{40FC4580-A84A-4E0C-9EB2-FC02EC3475C8},POINT (-75.721708 45.490631)
2,199,81017,112,Rue de Lacaune,Non Attribué,112 Rue de Lacaune,2638,{B2DE71BD-D4F6-4014-BC10-B34768C3CCF5},POINT (-75.721607 45.490147)
3,200,81017,111,Rue de Lacaune,Non Attribué,111 Rue de Lacaune,2638,{AF6656E2-C657-4F1E-9685-95A8B389C8D9},POINT (-75.721519 45.490596)
4,282,81017,87,Rue de Lacaune,Non Attribué,87 Rue de Lacaune,2638,{A0FBFDA5-B98B-4AAB-ACFA-92384FFB4640},POINT (-75.720391 45.490386)
...,...,...,...,...,...,...,...,...,...
96007,99811,81017,46,Rue Latour,Non Attribué,46 Rue Latour,2673,{8C808A0D-6059-474C-ADAE-3DC80BC3844B},POINT (-75.399655 45.58534)
96008,99810,81017,44,Rue Latour,Non Attribué,44 Rue Latour,2673,{AC8A472D-DE7F-4C8C-8FC6-280B3D0D9F73},POINT (-75.399667 45.585397)
96009,99814,81017,52,Rue Latour,Non Attribué,52 Rue Latour,2673,{DC16CC64-15D7-4B41-B2F9-433913202838},POINT (-75.399556 45.585349)
96010,99819,81017,779(2),Boulevard Maloney,Est,779(2) Boulevard Maloney Est,2147,{3E4B7F64-383A-4764-80FA-120FBE3DA902},POINT (-75.615936 45.483131)


In [420]:
data4.to_csv("dataset_cleaned/adresse_clean.csv") 

### Cleaning Adresse  for Postgres

In [421]:
data_adresse = pd.read_csv("dataset_cleaned/adresse_clean.csv")

In [422]:
data_adresse

,Unnamed: 0,codeid,munid,numero_civ,lieu,direction,adresse_complete,ruesid,entiteid,geom
0,0,197,81017,88,Rue de Lacaune,Non Attribué,88 Rue de Lacaune,2638,{DA0C3CB2-7B9F-4891-ABFB-8A35E152F59F},POINT (-75.720493 45.489893)
1,1,198,81017,115,Rue de Lacaune,Non Attribué,115 Rue de Lacaune,2638,{40FC4580-A84A-4E0C-9EB2-FC02EC3475C8},POINT (-75.721708 45.490631)
2,2,199,81017,112,Rue de Lacaune,Non Attribué,112 Rue de Lacaune,2638,{B2DE71BD-D4F6-4014-BC10-B34768C3CCF5},POINT (-75.721607 45.490147)
3,3,200,81017,111,Rue de Lacaune,Non Attribué,111 Rue de Lacaune,2638,{AF6656E2-C657-4F1E-9685-95A8B389C8D9},POINT (-75.721519 45.490596)
4,4,282,81017,87,Rue de Lacaune,Non Attribué,87 Rue de Lacaune,2638,{A0FBFDA5-B98B-4AAB-ACFA-92384FFB4640},POINT (-75.720391 45.490386)
...,...,...,...,...,...,...,...,...,...,...
96007,96007,99811,81017,46,Rue Latour,Non Attribué,46 Rue Latour,2673,{8C808A0D-6059-474C-ADAE-3DC80BC3844B},POINT (-75.399655 45.58534)
96008,96008,99810,81017,44,Rue Latour,Non Attribué,44 Rue Latour,2673,{AC8A472D-DE7F-4C8C-8FC6-280B3D0D9F73},POINT (-75.399667 45.585397)
96009,96009,99814,81017,52,Rue Latour,Non Attribué,52 Rue Latour,2673,{DC16CC64-15D7-4B41-B2F9-433913202838},POINT (-75.399556 45.585349)
96010,96010,99819,81017,779(2),Boulevard Maloney,Est,779(2) Boulevard Maloney Est,2147,{3E4B7F64-383A-4764-80FA-120FBE3DA902},POINT (-75.615936 45.483131)


In [423]:
data_adresse = data_adresse.drop(columns = ["Unnamed: 0","munid"])

In [424]:
data_adresse = data_adresse.copy()

data_adresse[["longitude", "latitude"]] = data_adresse["geom"].str.extract(
    r"POINT \((-?\d+\.\d+)\s+(-?\d+\.\d+)\)"
)

data_adresse.loc[:, "longitude"] = data_adresse["longitude"].astype(float)
data_adresse.loc[:, "latitude"]  = data_adresse["latitude"].astype(float)

In [425]:
data_adresse

,codeid,numero_civ,lieu,direction,adresse_complete,ruesid,entiteid,geom,longitude,latitude
0,197,88,Rue de Lacaune,Non Attribué,88 Rue de Lacaune,2638,{DA0C3CB2-7B9F-4891-ABFB-8A35E152F59F},POINT (-75.720493 45.489893),-75.720493,45.489893
1,198,115,Rue de Lacaune,Non Attribué,115 Rue de Lacaune,2638,{40FC4580-A84A-4E0C-9EB2-FC02EC3475C8},POINT (-75.721708 45.490631),-75.721708,45.490631
2,199,112,Rue de Lacaune,Non Attribué,112 Rue de Lacaune,2638,{B2DE71BD-D4F6-4014-BC10-B34768C3CCF5},POINT (-75.721607 45.490147),-75.721607,45.490147
3,200,111,Rue de Lacaune,Non Attribué,111 Rue de Lacaune,2638,{AF6656E2-C657-4F1E-9685-95A8B389C8D9},POINT (-75.721519 45.490596),-75.721519,45.490596
4,282,87,Rue de Lacaune,Non Attribué,87 Rue de Lacaune,2638,{A0FBFDA5-B98B-4AAB-ACFA-92384FFB4640},POINT (-75.720391 45.490386),-75.720391,45.490386
...,...,...,...,...,...,...,...,...,...,...
96007,99811,46,Rue Latour,Non Attribué,46 Rue Latour,2673,{8C808A0D-6059-474C-ADAE-3DC80BC3844B},POINT (-75.399655 45.58534),-75.399655,45.58534
96008,99810,44,Rue Latour,Non Attribué,44 Rue Latour,2673,{AC8A472D-DE7F-4C8C-8FC6-280B3D0D9F73},POINT (-75.399667 45.585397),-75.399667,45.585397
96009,99814,52,Rue Latour,Non Attribué,52 Rue Latour,2673,{DC16CC64-15D7-4B41-B2F9-433913202838},POINT (-75.399556 45.585349),-75.399556,45.585349
96010,99819,779(2),Boulevard Maloney,Est,779(2) Boulevard Maloney Est,2147,{3E4B7F64-383A-4764-80FA-120FBE3DA902},POINT (-75.615936 45.483131),-75.615936,45.483131


In [426]:
url = "https://www.gatineau.ca/upload/donneesouvertes/DECOUPAGE_ADMINISTRATIF.json" 
gdf = gpd.read_file(url)
print(gdf.head())

   MUNID CODEID            NOM      TYPE ABREV          AIRE     PERIM  \
0  81017      5     Buckingham  Ex ville     B  1.608119e+07  18214.35   
1  81017     10  Masson-Angers  Ex ville     M  6.453902e+07  46386.20   
2  81017     15       Gatineau  Ex ville     G  1.585884e+08  58086.51   
3  81017     20           Hull  Ex ville     H  4.133173e+07  32973.84   
4  81017     25         Aylmer  Ex ville     A  1.000886e+08  42578.92   

                                 ENTITEID  \
0  {0DE4611D-2A9E-4B26-A1B6-72A9C3DD3DBD}   
1  {96AAFDBC-9949-41EF-8D00-ECE15E302886}   
2  {07DBC407-0068-4AEE-8F5E-51A985940380}   
3  {36908290-4C89-41DB-A00A-C29EB4CFB9A8}   
4  {5C1FA124-6734-44C6-AB31-23B28DC0EF91}   

                                            geometry  
0  POLYGON ((-75.41524 45.59898, -75.41522 45.598...  
1  POLYGON ((-75.35372 45.5957, -75.35272 45.5878...  
2  POLYGON ((-75.53744 45.57731, -75.5331 45.5479...  
3  POLYGON ((-75.77524 45.48664, -75.7751 45.4862...  
4  POLYGO

In [427]:
geometry = [Point(xy) for xy in zip(data_adresse["longitude"], data_adresse["latitude"])]
gdf_points = gpd.GeoDataFrame(data_adresse, geometry=geometry, crs="EPSG:4326")

In [428]:
gdf = gdf.to_crs("EPSG:4326")

In [429]:
geo3 = gpd.sjoin(gdf_points, gdf, how="left", predicate="within")

In [430]:
geo3

,codeid,numero_civ,lieu,direction,adresse_complete,ruesid,entiteid,geom,longitude,latitude,geometry,index_right,MUNID,CODEID,NOM,TYPE,ABREV,AIRE,PERIM,ENTITEID
0,197,88,Rue de Lacaune,Non Attribué,88 Rue de Lacaune,2638,{DA0C3CB2-7B9F-4891-ABFB-8A35E152F59F},POINT (-75.720493 45.489893),-75.720493,45.489893,POINT (-75.72049 45.48989),2,81017,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
1,198,115,Rue de Lacaune,Non Attribué,115 Rue de Lacaune,2638,{40FC4580-A84A-4E0C-9EB2-FC02EC3475C8},POINT (-75.721708 45.490631),-75.721708,45.490631,POINT (-75.72171 45.49063),2,81017,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
2,199,112,Rue de Lacaune,Non Attribué,112 Rue de Lacaune,2638,{B2DE71BD-D4F6-4014-BC10-B34768C3CCF5},POINT (-75.721607 45.490147),-75.721607,45.490147,POINT (-75.72161 45.49015),2,81017,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
3,200,111,Rue de Lacaune,Non Attribué,111 Rue de Lacaune,2638,{AF6656E2-C657-4F1E-9685-95A8B389C8D9},POINT (-75.721519 45.490596),-75.721519,45.490596,POINT (-75.72152 45.4906),2,81017,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
4,282,87,Rue de Lacaune,Non Attribué,87 Rue de Lacaune,2638,{A0FBFDA5-B98B-4AAB-ACFA-92384FFB4640},POINT (-75.720391 45.490386),-75.720391,45.490386,POINT (-75.72039 45.49039),2,81017,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96007,99811,46,Rue Latour,Non Attribué,46 Rue Latour,2673,{8C808A0D-6059-474C-ADAE-3DC80BC3844B},POINT (-75.399655 45.58534),-75.399655,45.58534,POINT (-75.39966 45.58534),0,81017,5,Buckingham,Ex ville,B,1.608119e+07,18214.35,{0DE4611D-2A9E-4B26-A1B6-72A9C3DD3DBD}
96008,99810,44,Rue Latour,Non Attribué,44 Rue Latour,2673,{AC8A472D-DE7F-4C8C-8FC6-280B3D0D9F73},POINT (-75.399667 45.585397),-75.399667,45.585397,POINT (-75.39967 45.5854),0,81017,5,Buckingham,Ex ville,B,1.608119e+07,18214.35,{0DE4611D-2A9E-4B26-A1B6-72A9C3DD3DBD}
96009,99814,52,Rue Latour,Non Attribué,52 Rue Latour,2673,{DC16CC64-15D7-4B41-B2F9-433913202838},POINT (-75.399556 45.585349),-75.399556,45.585349,POINT (-75.39956 45.58535),0,81017,5,Buckingham,Ex ville,B,1.608119e+07,18214.35,{0DE4611D-2A9E-4B26-A1B6-72A9C3DD3DBD}
96010,99819,779(2),Boulevard Maloney,Est,779(2) Boulevard Maloney Est,2147,{3E4B7F64-383A-4764-80FA-120FBE3DA902},POINT (-75.615936 45.483131),-75.615936,45.483131,POINT (-75.61594 45.48313),2,81017,15,Gatineau,Ex ville,G,1.585884e+08,58086.51,{07DBC407-0068-4AEE-8F5E-51A985940380}


In [431]:
geo3 = geo3.drop(columns=[
    'geometry',
    'index_right',
    'MUNID',
    'NOM',
    'TYPE',
    'ABREV',
    'AIRE',
    'PERIM',
    'ENTITEID'
])

In [432]:
geo3

,codeid,numero_civ,lieu,direction,adresse_complete,ruesid,entiteid,geom,longitude,latitude,CODEID
0,197,88,Rue de Lacaune,Non Attribué,88 Rue de Lacaune,2638,{DA0C3CB2-7B9F-4891-ABFB-8A35E152F59F},POINT (-75.720493 45.489893),-75.720493,45.489893,15
1,198,115,Rue de Lacaune,Non Attribué,115 Rue de Lacaune,2638,{40FC4580-A84A-4E0C-9EB2-FC02EC3475C8},POINT (-75.721708 45.490631),-75.721708,45.490631,15
2,199,112,Rue de Lacaune,Non Attribué,112 Rue de Lacaune,2638,{B2DE71BD-D4F6-4014-BC10-B34768C3CCF5},POINT (-75.721607 45.490147),-75.721607,45.490147,15
3,200,111,Rue de Lacaune,Non Attribué,111 Rue de Lacaune,2638,{AF6656E2-C657-4F1E-9685-95A8B389C8D9},POINT (-75.721519 45.490596),-75.721519,45.490596,15
4,282,87,Rue de Lacaune,Non Attribué,87 Rue de Lacaune,2638,{A0FBFDA5-B98B-4AAB-ACFA-92384FFB4640},POINT (-75.720391 45.490386),-75.720391,45.490386,15
...,...,...,...,...,...,...,...,...,...,...,...
96007,99811,46,Rue Latour,Non Attribué,46 Rue Latour,2673,{8C808A0D-6059-474C-ADAE-3DC80BC3844B},POINT (-75.399655 45.58534),-75.399655,45.58534,5
96008,99810,44,Rue Latour,Non Attribué,44 Rue Latour,2673,{AC8A472D-DE7F-4C8C-8FC6-280B3D0D9F73},POINT (-75.399667 45.585397),-75.399667,45.585397,5
96009,99814,52,Rue Latour,Non Attribué,52 Rue Latour,2673,{DC16CC64-15D7-4B41-B2F9-433913202838},POINT (-75.399556 45.585349),-75.399556,45.585349,5
96010,99819,779(2),Boulevard Maloney,Est,779(2) Boulevard Maloney Est,2147,{3E4B7F64-383A-4764-80FA-120FBE3DA902},POINT (-75.615936 45.483131),-75.615936,45.483131,15


In [433]:
geo3.duplicated().sum()

0

In [434]:
geo3_dict = {
    "codeid": "adresseid",
    "numero_civ": "numero_civ",
    "lieu": "lieu",
    'direction': 'direction',
    "adresse_complete": "adresse_complete",
    "ruesid": "ruesid",
    "entiteid": "entiteid",
    "geom":"geom",
    "longitude": "longitude",
    "latitude": "latitude",
    "CODEID": "decoupageid"
}

geo3 = geo3.rename(columns=geo3_dict)


In [435]:
geo3.dtypes

adresseid            int64
numero_civ          object
lieu                object
direction           object
adresse_complete    object
ruesid               int64
entiteid            object
geom                object
longitude           object
latitude            object
decoupageid         object
dtype: object

In [436]:
geo3['latitude'] = pd.to_numeric(geo3['latitude'], errors='coerce')
geo3['longitude'] = pd.to_numeric(geo3['longitude'], errors='coerce')
geo3['numero_civ'] = pd.to_numeric(geo3['numero_civ'], errors='coerce').astype('Int64')
geo3['decoupageid'] = pd.to_numeric(geo3['decoupageid'], errors='coerce').astype('Int64')



In [437]:
geo3.isna().sum()

adresseid              0
numero_civ          1339
lieu                   0
direction              0
adresse_complete       0
ruesid                 0
entiteid               0
geom                   0
longitude              0
latitude               0
decoupageid            0
dtype: int64

In [438]:
geo3['numero_civ'] = geo3['numero_civ'].fillna(0)

In [439]:
geo3["numero_civ"].isna().sum()

0

In [440]:
geo3.to_csv("pg_cleaned/adresse_geo.csv", index= False)